In [1]:
from init import *
import time

In [2]:
model = Model()


In [3]:
m = mcsdk.examples.BuildModel_Simple()
m.evaluations.append(EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentBrownianMotion"}, {"name": "BrownianMotion", "start": {"name": "start", "value": 0.1}, "args": [{"name": "drift", "value": 0.2}, {"name": "diffusion", "value": 2}]}], "evaluations": [{"state": 0, "time": 2}]}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'mean': [0.5230804085731506],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'skewness': [-0.01593666523694992],
 'stddev': [2.853414535522461],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
m = mcsdk.examples.BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 10000
for t in [1,2,5,10]:
    m.evaluations.append(EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'mean': [0.2997841536998749,
  0.5001057982444763,
  1.0959651470184326,
  2.0978493690490723],
 'names': ['BrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'skewness': [0.0025370384100824594,
  -0.0003037625865545124,
  -6.074928387533873e-05,
  -0.0019319666316732764],
 'stddev': [2.000171661376953,
  2.826313018798828,
  4.471774101257324,
  6.330868244171143],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [1000, 2000, 5000, 10000]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

3.0 nanoseconds to make a single time step


In [7]:
m = mcsdk.examples.BadModel()
m.NumPaths = 1000000
m.TimeSteps = 10000
for t in [1,2,5,10]:
    m.evaluations.append(EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'error': ['../src/binding/json/all.cpp void from_json(const json&, Model&) AddEvaluationPoint']}

# Two processes, Three evaluations

In [8]:
model = mcsdk.examples.TwoProcessesModel()
model.TimeSteps = 10000
for time in [1,2,5]:
    model.evaluations.append(EvaluationPoint(state=0,time=time))
r = requests.post(f'{server}/model',model.json())
r.json()

{'mean': [0.30169379711151123,
  0.2891586124897003,
  0.5055257081985474,
  0.4809056520462036,
  1.0719531774520874,
  1.1176645755767822],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'skewness': [-0.008076229132711887,
  0.015258703380823135,
  -0.0010828811209648848,
  0.019505947828292847,
  0.05010344460606575,
  0.03506255894899368],
 'stddev': [1.9988831281661987,
  1.9962321519851685,
  2.8082897663116455,
  2.7867650985717773,
  4.505069255828857,
  4.473954200744629],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [9]:
er = EvaluationResults(r.json(),model)
er

2 states with 3 evaluations

In [10]:
er.df()

,name,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,0,1.0,2000,10000,0.301694,0.019989,1.998883,-0.008076
1,BrownianMotion,1,1.0,2000,10000,0.289159,0.019962,1.996232,0.015259
2,BrownianMotion,0,2.0,4000,10000,0.505526,0.028083,2.808290,-0.001083
3,BrownianMotion,1,2.0,4000,10000,0.480906,0.027868,2.786765,0.019506
4,BrownianMotion,0,5.0,10000,10000,1.071953,0.045051,4.505069,0.050103
5,BrownianMotion,1,5.0,10000,10000,1.117665,0.044740,4.473954,0.035063
